In [3]:
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-d6cc  GCE       2                                       RUNNING  us-central1-f
NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-d6cc  GCE       2                                       RUNNING  us-central1-f


In [4]:
# !pip install -q google-cloud-storage==1.43.0
# !pip install -q graphframes

In [5]:
# !pip install nltk==3.7
# !python -m nltk.downloader wordnet

In [6]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
# from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from math import log, pow
from google.cloud import storage
from nltk.stem import WordNetLemmatizer, PorterStemmer

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data]   Package omw-1.4 is already up-to-date!


True

True

In [7]:
import findspark
findspark.init() 
from pyspark.sql import *
import pyspark.sql.functions as pyFunc
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
# from graphframes import *

In [8]:
bucket_name = '201640042_project' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.endswith('parquet'):
        paths.append(full_path+b.name)

In [9]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd
doc_title_pairs = parquetFile.select("title", "id").rdd
doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd
n_pages = parquetFile.count()

In [10]:
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [11]:
from inverted_index_gcp import *

In [12]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124

# # # Opening JSON file
# import json
# with open('/home/dataproc/queries_train.json') as json_file:
#     data = json.load(json_file)
# train_tokens = [token.group() for token in RE_WORD.finditer(' '.join(data.keys()).lower())]
# train_filtered_vocab = set([lemmatizer.lemmatize(tok) for tok in train_tokens if (tok not in all_stopwords)])

def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS


def get_tokens(text, doc_id):
    """Support function - performs token filtering per doc 
      Parameters:
    -----------
    text: str
      Text of one document
    id: int
      Document id
    Returns:
    --------
    List of tuples
      A list of (doc_id, filtered_token_list) pairs 
    """
    tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if token != '']
    filtered_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if (tok not in all_stopwords)]
    return [(doc_id, filtered_tokens)]


def word_count(doc_id, tokens):
    """ Count the frequency of each word in `text` (tf) that is not included in 
    `all_stopwords` and return entries that will go into our posting lists. 
    Parameters:
    -----------
    id: int
      Document id
    tokens: str
      list of tokens from document
    Returns:
    --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
    """
    doc_word_count = Counter(tokens)
    return [(tok, (doc_id, tf)) for tok, tf in doc_word_count.items()]

def get_doc_len(doc_id, tokens):
    """ Count document filtered length for storage in index as well as document vector length for RDD calculations
  Parameters:
  -----------
    id: int
      Document id
    tokens: str
      list of tokens from document
    Returns:
  --------
    List of tuples
      A list of (doc_id, doc_length) pairs
  """
    doc_word_count = Counter(tokens)
    doc_len = sum(doc_word_count.values())
    return [(doc_id, doc_len)]
    

def get_tfidf(tf, df, doc_len):
    if doc_len == 0:
        return 0.0
    tf_idf = tf/doc_len * log(n_pages / (df + 1), 2)
    return tf_idf

In [13]:
# get title dict for inverted index to quickly prepare results
id2title = doc_title_pairs.map(lambda x: (x[1], x[0])).collectAsMap()

In [ ]:
# rdd_dict = {"full_body_index": doc_text_pairs, "full_title_index": doc_title_pairs, "full_anchor_index": doc_anchor_pairs}
rdd_dict = {"full_lemmatized_body_index": doc_text_pairs}
for rdd_name, rdd_pairs in rdd_dict.items():
    # for anchors - need to handle duplicated anchors found on different pages pointing to the same page with similar or different anchor text
    if (rdd_name == "full_lemmatized_anchor_index"):
        print("running anchorIndex")
        doc_tok = rdd_pairs.flatMap(lambda x: x[0]).flatMap(lambda x: get_tokens(x[1], x[0])).reduceByKey(lambda x,y: x+y).mapValues(set)
    else:
        doc_tok = rdd_pairs.flatMap(lambda x: get_tokens(x[0], x[1]))
    
    # calculate document length for later tf normalization
    doc_length = doc_tok.flatMap(lambda x: get_doc_len(x[0], x[1]))
    # calculate term frequency by document
    word_counts = doc_tok.flatMap(lambda x: word_count(x[0], x[1]))
    if (rdd_name == "full_lemmatized_body_index"):
        print("running bodyIndex with uncommon_words filter\n")
        postings = word_counts.groupByKey().mapValues(reduce_word_counts).filter(lambda x: len(x[1]) >= 50)
    else:
        postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    # Calculate term document frequency
    w2df = calculate_df(postings)
    # Calculate norm of each document - get tf from posting, df from w2df, doc length and calculate tfidf^2 per doc_id, term and sum by doc_id
    doc2norm = postings.flatMapValues(lambda x: x).leftOuterJoin(w2df) \
        .map(lambda x: (x[1][0][0], (x[0], x[1][0][1], x[1][1]))) \
        .leftOuterJoin(doc_length).map(lambda x: (x[0], (x[1][0][1], x[1][0][2], x[1][1]))) \
        .mapValues(lambda x: pow(get_tfidf(x[0], x[1], x[2]), 2)) \
        .reduceByKey(lambda x,y: x+y).mapValues(lambda x: round(x, 6))

    # save doc_id: (doc_norm, doc_len) RDD for cosin similarity calculations
    doc_data = doc2norm.join(doc_length)

    # write posting to bin files
    _ = partition_postings_and_write(postings, bucket_name, rdd_name).collect()
    print(f"PostingLocs created for {rdd_name}\n")
    
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix=rdd_name):
        if not blob.name.endswith("pickle"):
            continue
        with blob.open("rb") as f:
            posting_locs = pickle.load(f)
            for k, v in posting_locs.items():
                super_posting_locs[k].extend(v)
                
    # Create inverted index instance
    inverted = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    inverted.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted.df.update(w2df.collectAsMap())
    print(f"document frequancy created for {rdd_name}\n")
    # Count number of docs
    inverted._N = n_pages
    # Get each document length and norm
    inverted.doc_data.update(doc_data.collectAsMap())
    print(f"doc data created for {rdd_name}\n")
    # save titles to return results
    inverted.doc2title = id2title
    # write the global stats out
    inverted.write_index('.', rdd_name)
    print(f"{rdd_name} written\n")
    
    # upload to gs
    index_src = f"{rdd_name}.pkl"
    index_dst = f'gs://{bucket_name}/{rdd_name}/{index_src}'
    !gsutil cp $index_src $index_dst
    print(f"{rdd_name} uploaded to bucket\n")
    
    !gsutil ls -lh $index_dst

running bodyIndex with uncommon_words filter

running bodyIndex with uncommon_words filter



PostingLocs created for full_lemmatized_body_index



document frequancy created for full_lemmatized_body_index



doc data created for full_lemmatized_body_index

full_lemmatized_body_index written

Copying file://full_lemmatized_body_index.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][297.8 MiB/297.8 MiB]                                                
Operation completed over 1 objects/297.8 MiB.              